## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mar Del Plata,AR,2021-09-27 01:18:41,-38.0023,-57.5575,54.09,66,90,4.00,overcast clouds
1,1,Kapaa,US,2021-09-27 01:19:10,22.0752,-159.3190,84.18,73,75,5.01,light rain
2,2,Mataura,NZ,2021-09-27 01:18:39,-46.1927,168.8643,51.48,35,6,2.44,clear sky
3,3,Sao Filipe,CV,2021-09-27 01:19:10,14.8961,-24.4956,78.58,80,87,5.23,overcast clouds
4,4,Egvekinot,RU,2021-09-27 01:19:11,66.3167,-179.1667,26.69,71,9,2.80,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_selected_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
cities_selected_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kapaa,US,2021-09-27 01:19:10,22.0752,-159.3190,84.18,73,75,5.01,light rain
3,3,Sao Filipe,CV,2021-09-27 01:19:10,14.8961,-24.4956,78.58,80,87,5.23,overcast clouds
6,6,Pangai,TO,2021-09-27 01:19:12,-19.8000,-174.3500,78.91,47,75,5.75,broken clouds
7,7,Manati,PR,2021-09-27 01:19:12,18.4275,-66.4921,83.86,89,38,6.71,scattered clouds
13,13,Formello,IT,2021-09-27 01:19:14,42.0801,12.3998,73.11,76,33,2.13,scattered clouds
18,18,Rikitea,PF,2021-09-27 01:19:16,-23.1203,-134.9692,73.78,72,2,19.42,clear sky
21,21,Sabha,LY,2021-09-27 01:19:17,27.0377,14.4283,71.42,30,0,8.88,clear sky
22,22,Markala,ML,2021-09-27 01:19:17,13.7021,-6.0659,79.41,63,26,10.45,scattered clouds
23,23,La Romana,DO,2021-09-27 01:19:18,18.4273,-68.9728,80.33,94,75,0.00,broken clouds
25,25,Ponta Do Sol,PT,2021-09-27 01:19:18,32.6667,-17.1000,70.43,68,27,3.36,scattered clouds


In [10]:
# 4a. Determine if there are any empty rows.
cities_selected_df.notnull().sum()

City_ID                294
City                   294
Country                294
Date                   294
Lat                    294
Lng                    294
Max Temp               294
Humidity               294
Cloudiness             294
Wind Speed             294
Current Description    294
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_selected_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,84.18,light rain,22.0752,-159.3190,
3,Sao Filipe,CV,78.58,overcast clouds,14.8961,-24.4956,
6,Pangai,TO,78.91,broken clouds,-19.8000,-174.3500,
7,Manati,PR,83.86,scattered clouds,18.4275,-66.4921,
13,Formello,IT,73.11,scattered clouds,42.0801,12.3998,
18,Rikitea,PF,73.78,clear sky,-23.1203,-134.9692,
21,Sabha,LY,71.42,clear sky,27.0377,14.4283,
22,Markala,ML,79.41,scattered clouds,13.7021,-6.0659,
23,La Romana,DO,80.33,broken clouds,18.4273,-68.9728,
25,Ponta Do Sol,PT,70.43,scattered clouds,32.6667,-17.1000,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
print("------------------------------------")
print("Start hotel analysis")

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
        
    # 6c. Get latitude and longitude from DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    params["location"] = f"{latitude},{longitude}"
      
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Not info for this hotel!")

print("------------------------------------")            
print("End of hotel research")       

------------------------------------
Start hotel analysis
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
Not info for this hotel!
------------------------------------
End of hotel research


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,84.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
3,Sao Filipe,CV,78.58,overcast clouds,14.8961,-24.4956,Hotel Xaguate
6,Pangai,TO,78.91,broken clouds,-19.8000,-174.3500,Ha'apai Beach Resort
7,Manati,PR,83.86,scattered clouds,18.4275,-66.4921,Manati World Point Inn
13,Formello,IT,73.11,scattered clouds,42.0801,12.3998,Antica Pietrara
...,...,...,...,...,...,...,...
669,Sahuaripa,MX,81.90,clear sky,29.0500,-109.2333,Estatua de Sahuaripa
672,Umarkot,PK,84.76,clear sky,25.3616,69.7362,Al-Qamar Boys Hostel
673,Pacific Grove,US,71.62,broken clouds,36.6177,-121.9166,Lovers Point Inn
676,Limbe,CM,74.44,overcast clouds,4.0242,9.2149,The LK Hotel at Bonadikombo


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))